In [ ]:
import os,glob
import pandas as pd
import numpy as np

change country between: PR, DR, CU

In [ ]:
user = "Chris"
country = "CU"

In [ ]:
CU_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/cuba"
DR_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom"
PR_drectory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico"

if user == "Chris":
    if country == "DR":
        current_directory = DR_directory
        path1 = DR_directory+"/All_CSVs" #where all CSVs are located
        path2 = DR_directory+"/Sync Files/06.11.23/CSV" #change to show latest upload
    elif country == "CU":
        current_directory = CU_directory
        path1 = CU_directory+"/All_CSVs" #where we merge all cumulative CSV files into one file
        path2 = CU_directory+"/10_17_23/CSV" #where we check to see that the latest batch of data includes all files it should
    elif country == "PR":
        current_directory = PR_directory
        path1 = PR_directory+"/All_CSVs"
        path2 = PR_directory+"/11_13_23"
elif user == "Ty":
    current_directory = r"C:\Users\Ty\Desktop\DataExport"
else:
    print("Unknown user. Cannot set directory.")
    current_directory = None

if current_directory:
    os.chdir(current_directory)
    file_list = os.listdir(current_directory)
    print(f"Changed directory to: {current_directory}")
    print(path1)
    print(path2)
    print(file_list)

Received files document

In [ ]:
# Get the list of all files in the latest file upload
dir_list1 = os.listdir(path2)

# To see which files I received from the data manager
newdir_list1 = [x for x in dir_list1 if x.endswith('.epi7')]

# To see which files I was able to process with Epi Info tool
dir_list2 = os.listdir(path2)
newdir_list2 = [x for x in dir_list2 if x.endswith('.csv')]

def remove_common(a, b):
    for i in a[:]:
        if i in b:
            a.remove(i)
            b.remove(i)

    print("list1 : ", a)
    print("list2 : ", b)

remove_common(newdir_list1, newdir_list2)

words_to_filter = ["Neighborhood", "Door", "_informationdoor_", "_informationdoorparticipants", "Listas", "_mainhousehold", "_participants", "_rosterhousehold", "_nonresidentchildren", "Cognitva", "_cognitive", "Cognitive_Scoring", "Examen_Físico", "_physical_exam","Familiar", "_household", "Informante", "_informant"]

# breaking the longer list into smaller lists
filtered_lists = {word: [] for word in words_to_filter}

for text in newdir_list2:
    for word in words_to_filter:
        if word in text:
            filtered_lists[word].append(text)

# change this number depending on how many tablets worth of information you're receiving
for word, filtered_list in filtered_lists.items():
    while len(filtered_list) < 6:
        filtered_list.append("Falta")

Received_Files = pd.DataFrame()

for word, filtered_list in filtered_lists.items():
    Received_Files[word] = pd.Series(filtered_list)

Received_Files.to_excel('Received_Files_Checklist.xlsx', index=False)

Received_Files

In [ ]:
if user == "Chris":
    if country == "CU":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/cuba/All_CSVs/"
    elif country == "DR":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/Rep \Dom/All_CSVs/"
    elif country == "PR":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/"
elif user == "Ty":
    if country == "CU":
        pattern = r"C:\Users\Ty\Desktop\DataExport"
    elif country == "DR":
        pattern = r"C:\Users\Ty\Desktop\DataExport"
else:
    print("Unknown user. Cannot set directory.")
    current_directory = None

folder_path = path1 #goes through this whole folder

####COGNITIVE CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_cognitive*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + "_cognitive_"
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_child = pd.concat(dfs, ignore_index=True)
cog_child = cog_child.sort_values(by='filename', ascending=False)
cog_child = cog_child.drop(columns=['filename'])
cog_child = cog_child.drop_duplicates(subset='GlobalRecordId', keep='first')
pattern_to_remove = r'=HYPERLINK\("media\\([^"]+)",\"<CLICK HERE>"\)'
cog_child = cog_child.replace(pattern_to_remove, r'\1', regex=True)

cog_child.to_csv("CSV_Merged/Cog_Child.csv")

####COGNITIVE PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Cognitiva*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Cognitiva_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_parent = pd.concat(dfs, ignore_index=True)
cog_parent = cog_parent.sort_values(by='filename', ascending=False)
cog_parent = cog_parent.drop(columns=['filename'])
cog_parent = cog_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_parent.to_csv("CSV_Merged/Cog_Parent.csv")

####COGNITIVE SCORING#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Cognitive_Scoring*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Cognitive_Scoring_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_scoring = pd.concat(dfs, ignore_index=True)
cog_scoring = cog_scoring.sort_values(by='filename', ascending=False)
cog_scoring = cog_scoring.drop(columns=['filename'])
cog_scoring = cog_scoring.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_scoring.to_csv("CSV_Merged/Cog_Scoring.csv")

####HOUSEHOLD CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_household*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_household_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
household_child = pd.concat(dfs, ignore_index=True)
household_child = household_child.sort_values(by='filename', ascending=False)
household_child = household_child.drop(columns=['filename'])
household_child = household_child.drop_duplicates(subset='GlobalRecordId', keep='first')

household_child.to_csv("CSV_Merged/Household_Child.csv")

####HOUSEHOLD PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Familiar*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Familiar_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
household_parent = pd.concat(dfs, ignore_index=True)
household_parent = household_parent.sort_values(by='filename', ascending=False)
household_parent = household_parent.drop(columns=['filename'])
household_parent = household_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

household_parent.to_csv("CSV_Merged/Household_Parent.csv")

####PHYS CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_physical_exam*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_physical_exam_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
phys_child = pd.concat(dfs, ignore_index=True)
phys_child = phys_child.sort_values(by='filename', ascending=False)
phys_child = phys_child.drop(columns=['filename'])
phys_child = phys_child.drop_duplicates(subset='GlobalRecordId', keep='first')

phys_child.to_csv("CSV_Merged/Phys_Child.csv")

####PHYS PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Examen_Físico*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Examen_Físico_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
phys_parent = pd.concat(dfs, ignore_index=True)
phys_parent = phys_parent.sort_values(by='filename', ascending=False)
phys_parent = phys_parent.drop(columns=['filename'])
phys_parent = phys_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

phys_parent.to_csv("CSV_Merged/Phys_Parent.csv")

####INFOR CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informant*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informant_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
infor_child = pd.concat(dfs, ignore_index=True)
infor_child = infor_child.sort_values(by='filename', ascending=False)
infor_child = infor_child.drop(columns=['filename'])
infor_child = infor_child.drop_duplicates(subset='GlobalRecordId', keep='first')

infor_child.to_csv("CSV_Merged/Infor_Child.csv")

####INFOR PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Informante*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Informante_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
infor_parent = pd.concat(dfs, ignore_index=True)
infor_parent = infor_parent.sort_values(by='filename', ascending=False)
infor_parent = infor_parent.drop(columns=['filename'])
infor_parent = infor_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

infor_parent.to_csv("CSV_Merged/Infor_Parent.csv")

####SOCIO CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_sociodemographic*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_sociodemographic_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
socio_child = pd.concat(dfs, ignore_index=True)
socio_child = socio_child.sort_values(by='filename', ascending=False)
socio_child = socio_child.drop(columns=['filename'])
socio_child = socio_child.drop_duplicates(subset='GlobalRecordId', keep='first')

socio_child.to_csv("CSV_Merged/Socio_Child.csv")

####SOCIO PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Sociodemográfica*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Sociodemográfica_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
socio_parent = pd.concat(dfs, ignore_index=True)
socio_parent = socio_parent.sort_values(by='filename', ascending=False)
socio_parent = socio_parent.drop(columns=['filename'])
socio_parent = socio_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

socio_parent.to_csv("CSV_Merged/Socio_Parent.csv")

####NEIGHBORHOOD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Neighborhood*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Neighborhood_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
neighborhood = pd.concat(dfs, ignore_index=True)
neighborhood = neighborhood.sort_values(by='filename', ascending=False)
neighborhood = neighborhood.drop(columns=['filename'])
neighborhood = neighborhood.drop_duplicates(subset='GlobalRecordId', keep='first')

neighborhood.to_csv("CSV_Merged/Neighborhood.csv")

####DOOR PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Door*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Door_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
door_parent = pd.concat(dfs, ignore_index=True)
door_parent = door_parent.sort_values(by='filename', ascending=False)
door_parent = door_parent.drop(columns=['filename'])
door_parent = door_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

door_parent.to_csv("CSV_Merged/Door.csv")

####DOOR CHILD 1#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informationdoor_*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informationdoor_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
informationdoor = pd.concat(dfs, ignore_index=True)

informationdoor = informationdoor.drop_duplicates(subset='GlobalRecordId', keep='first')
informationdoor = informationdoor.sort_values(by='filename', ascending=False)
informationdoor = informationdoor.drop(columns=['filename'])
informationdoor.to_csv("CSV_Merged/InformationDoor.csv")

####DOOR CHILD 2#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informationdoorparticipants*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informationdoorparticipants_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
informationdoorparticipants = pd.concat(dfs, ignore_index=True)

informationdoorparticipants = informationdoorparticipants.drop_duplicates(subset='GlobalRecordId', keep='first')
informationdoorparticipants = informationdoorparticipants.sort_values(by='filename', ascending=False)
informationdoorparticipants = informationdoorparticipants.drop(columns=['filename'])
informationdoorparticipants.to_csv("CSV_Merged/InformationDoorParticipants.csv")

####ROSTER PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Listas*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Listas_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
rosters_parent = pd.concat(dfs, ignore_index=True)
rosters_parent = rosters_parent.sort_values(by='filename', ascending=False)
rosters_parent = rosters_parent.drop(columns=['filename'])
rosters_parent = rosters_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

rosters_parent.to_csv("CSV_Merged/Roster_Parent.csv")

####ROSTER CHILD 1#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_mainhousehold*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_mainhousehold_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
mainhousehold = pd.concat(dfs, ignore_index=True)
mainhousehold = mainhousehold.sort_values(by='filename', ascending=False)
mainhousehold = mainhousehold.drop(columns=['filename'])
mainhousehold = mainhousehold.drop_duplicates(subset='GlobalRecordId', keep='first')

mainhousehold.to_csv("CSV_Merged/MainHousehold.csv")

####ROSTER CHILD 2#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_participants*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_participants_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
participants = pd.concat(dfs, ignore_index=True)
participants = participants.sort_values(by='filename', ascending=False)
participants = participants.drop(columns=['filename'])
participants = participants.drop_duplicates(subset='GlobalRecordId', keep='first')

participants.to_csv("CSV_Merged/Participants.csv")

####ROSTER CHILD 3#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_rosterhousehold*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_rosterhousehold_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
nonparticipants = pd.concat(dfs, ignore_index=True)
nonparticipants = nonparticipants.sort_values(by='filename', ascending=False)
nonparticipantrs = nonparticipants.drop(columns=['filename'])
nonparticipants = nonparticipants.drop_duplicates(subset='GlobalRecordId', keep='first')

nonparticipants.to_csv("CSV_Merged/NonParticipants.csv")

####ROSTER CHILD 4#######

dfs = [] 

for filename in glob.glob(os.path.join(folder_path, '*_nonresidentchildren*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_nonresidentchildren_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
nonresidentchildren = pd.concat(dfs, ignore_index=True)
nonresidentchildren = nonresidentchildren.sort_values(by='filename', ascending=False)
nonresidentchildren = nonresidentchildren.drop(columns=['filename'])
nonresidentchildren = nonresidentchildren.drop_duplicates(subset='GlobalRecordId', keep='first')

nonresidentchildren.to_csv("CSV_Merged/NonResidentChildren.csv")

if country != "CU":
    dfs = []  # list to hold all dataframes
    for filename in glob.glob(os.path.join(folder_path, '*Sangre*.csv')):
        print(f"Processing {filename}")
        df = pd.read_csv(filename)
        df['filename'] = filename
        pattern_to_remove = pattern + 'Sangre_'
        df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
        df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
        df['filename'] = df['filename'].astype(int)
        dfs.append(df)
        
    sangre = pd.concat(dfs, ignore_index=True)
    sangre = sangre.sort_values(by='filename', ascending=False)
    sangre = sangre.drop(columns=['filename'])
    sangre = sangre.drop_duplicates(subset='GlobalRecordId', keep='first')
    
    sangre.to_csv("CSV_Merged/Sangre.csv")